In [ ]:
!pip3 install -r requirements.txt

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter

from modules.dataset import IntelImageClassificationDataset
from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time
from modules.trainer import Trainer

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = IntelImageClassificationDataset(resize=(150,150))

https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [3]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

In [4]:
NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

BoundedIntText(value=0, description='item:', max=14033)

In [5]:
# SqueezeNet 1.0
# model = models.squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)
# num_features = model.classifier[1].in_channels
# kernel_size = model.classifier[1].kernel_size
# model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)

# SqueezeNet 1.1
model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
num_features = model.classifier[1].in_channels
kernel_size = model.classifier[1].kernel_size
model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)



dataloader = DataLoader(dataset.train_dataset, batch_size=128, shuffle=True)
trainer = Trainer(model=model, lr=0.001)

In [6]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))


trainer.train(dataloader, epochs=10)

In [7]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

0.9129151701927185

In [8]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

## Initial Results for Model Selection

| model | accuracy | size |
| --- | --- | --- |
| ResNet18 | 0.87 | 44.7 MB |
| ResNet34 | 0.88 | 83.3 MB |
| MobileNet V2 | 0.91 | 13.6 MB |
| MobileNet V3 small | 0.90 | 9.8 MB |
| VGG19 | 0.83 | 548.1 MB |
| SqueezeNet 1.0 | 0.89 | 4.8 MB |
| DenseNet | 0.90 | 30.8 MB |
| EfficientNet B0 | 0.92 | 20.5 MB |
| ViT-b/16 | 0.73 | 330.3 MB |